# POC

Methodology for POC:

1. Choose "AAPL".
2. Scrape news (from Yahoo Finance), for Apple stock via scrapling
3. Use ChatGPT to determine the sentiment of each news on scale of 1 to 5:

- 1 -> Negative
- 2 -> Moderate negative
- 3 -> Neutral
- 4 -> Moderate positive
- 5 -> Positive

4. Use ChatGPT to determine the stock that has highest cointegration with Apple

5. Use Selenium to load more news article


# Table of Contents

1. [Select Stocks](#select-stocks)
2. [Extract News](#extract-news)
3. [Perplexity Results](#perplexity-results-gpt-4o)
   - 3.1 [Prompt](#prompt)
   - 3.2 [Response](#response)
   - 3.3 [Cointegration](#cointegration)
   - 3.4 [Correlation](#correlation)
4. [Selenium](#selenium)
5. [Playwright](#playwright)


In [98]:
import pandas as pd
import random
import sys
from pathlib import Path
import scrapling
from scrapling import Fetcher
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np

repo_dir = Path.cwd().as_posix()

if repo_dir not in sys.path:
    sys.path.append(repo_dir)

%load_ext autoreload
%autoreload 2

from src.utils import utils, yahoo_utils
from src.gen_data import GenData
from src.sentiment import SentimentRater
from src.cointegrate import CoIntegrate


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# [Select Stocks](#table-of-contents)

- Test "AAPL", "NVDA", and "PG" for POC.
- If time permits, then test the randomly selected stocks for each GICS Sector: ['PWR', 'INCY', 'CDW', 'CEG', 'BAC', 'LIN', 'BWA', 'ARE', 'CHTR', 'KDP', 'PSX']


In [33]:
stock_list = utils.gen_stock_list()
stock_list

{'Industrials': 'PWR',
 'Health Care': 'INCY',
 'Information Technology': 'CDW',
 'Utilities': 'CEG',
 'Financials': 'BAC',
 'Materials': 'LIN',
 'Consumer Discretionary': 'BWA',
 'Real Estate': 'ARE',
 'Communication Services': 'CHTR',
 'Consumer Staples': 'KDP',
 'Energy': 'PSX'}

In [36]:
utils.get_gics_sector(
    ["AAPL", "MSFT", "NVDA", "GOOGL", "META", "XOM", "CRM", "HD", "PG"]
)

{'AAPL': 'Information Technology',
 'MSFT': 'Information Technology',
 'NVDA': 'Information Technology',
 'GOOGL': 'Communication Services',
 'META': 'Communication Services',
 'XOM': 'Energy',
 'CRM': 'Information Technology',
 'HD': 'Consumer Discretionary',
 'PG': 'Consumer Staples'}

# [Extract News](#table-of-contents)

- https://finance.yahoo.com/quote/AAPL/news/
- Scrapling aren't able to web scrap fully since the news are loaded when user scroll to the end of screen.


In [98]:
# Extract news related to AAPL from Yahoo Finance
news_info = yahoo_utils.extract_news_briefs(
    "https://finance.yahoo.com/quote/AAPL/news/"
)
news_info

[2025-03-08 21:01:29] INFO: Fetched (200) <GET https://finance.yahoo.com/quote/AAPL/news/> (referer: https://www.google.com/search?q=yahoo)


[{'publisher': 'Motley Fool',
  'period': datetime.timedelta(seconds=7200),
  'title': 'Could Apple Be a Millionaire-Maker Stock?',
  'content': 'During its rise to become a dominant tech enterprise, Apple (NASDAQ: AAPL) has certainly made its early investors some serious money. In the past two decades, shares have generated a total return of greater than 18,000%, boosted by the introduction of popular hardware devices and services. Could Apple be a millionaire-maker stock?'},
 {'publisher': 'Insider Monkey',
  'period': datetime.timedelta(seconds=43200),
  'title': 'Apple Inc. (AAPL) Unveils AI-Powered MacBook Air with M4 Chip',
  'content': 'We recently compiled a list of the 10 Hot AI News Updates Analysts are Monitoring. In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against the other AI stocks. According to Goldman Sachs, the AI-powered rally in Chinese equities could boost other markets as well. Consequently, the firm has […]'},
 {'publisher

# [Perplexity Results (GPT-4o)](#table-of-contents)


## [Prompt](#table-of-contents)

The list of dictionaries are attached with the following prompt in Perplexity:

```
You are a stock analysis expert.

I've attached list of dictionaries containing news info specifically:

1. publisher = Source where news are generated
2. period = Amount of time lapsed after news release
3. title = News title
4. content = News content in brief

Each dictionary is supposed to be related to Apple stock.

Your task:
1. Review the 'title' and 'content' keys in each dictionary to access the sentiment related to Apple stock.
2. Use ONLY following rating:

    1 = Negative
    2 = Moderate negative
    3 = Neutral
    4 = Moderate positive
    5 = Positive

3. If the news are not related to a Apple stock, please give a rating of 3.
4. Please review all dictionary items in the attached list and provide the corresponding rating.
5. After reviewing all dictionary items, kindly provide an overall sentiment rating for Apple stock.
```


## [Response](#table-of-contents)

Response from Perplexity (ChatGPT 4-o):

```
Sentiment Ratings for Each News Item
Title: "Could Apple Be a Millionaire-Maker Stock?"
Content: Highlights Apple's historical success and potential for future gains.
Rating: 5 (Positive)

Title: "Apple Inc. (AAPL) Unveils AI-Powered MacBook Air with M4 Chip"
Content: Discusses the launch of an innovative product, indicating progress in AI.
Rating: 5 (Positive)

Title: "Apple (AAPL) Beats Stock Market Upswing: What Investors Need to Know"
Content: Reports a positive trading session for Apple stock.
Rating: 4 (Moderate positive)

Title: "Apple Delays Siri Upgrade Indefinitely as AI Concerns Escalate"
Content: Focuses on delays in Apple's AI projects, raising concerns.
Rating: 2 (Moderate negative)

Title: "Apple Is Pushing Back AI-Driven Siri Features to 2026, Report Says"
Content: Similar to the previous item, emphasizes delays in Siri enhancements.
Rating: 2 (Moderate negative)

Title: "Apple Says Some AI-Powered Enhancements to Siri to Be Delayed"
Content: Mentions delays in Siri updates amid slumping iPhone sales.
Rating: 2 (Moderate negative)

Title: "Sector Update: Tech Stocks Advance Late Afternoon"
Content: General update on tech stocks without specific focus on Apple.
Rating: 3 (Neutral)

Title: "Magnificent Seven Stocks: Nvidia Reverses Higher; Amazon, Meta, Tesla Slide"
Content: Discusses multiple tech stocks, with a brief mention of Apple’s mixed performance in 2025.
Rating: 3 (Neutral)

Title: "Apple says some AI improvements to Siri delayed to 2026"
Content: Reiterates delays in Siri's AI features, consistent with earlier reports.
Rating: 2 (Moderate negative)

Title: "Apple's First Foldable iPhone Could Launch in 2026 With Premium Pricing and AI Features"
Content: Speculates on a high-priced foldable iPhone with advanced features, suggesting innovation but no immediate impact.
Rating: 4 (Moderate positive)

Title: "Apple Inc. (AAPL): Jim Cramer Cautions – ‘I Could See the Stock Going Lower!’"
Content: Warns of potential downside for Apple stock, reflecting negativity.
Rating: 1 (Negative)

Title: "Apple Secures Local Content Certifications in Indonesia but Faces Additional Regulatory Hurdles"
Content: Notes progress in regulatory compliance but highlights challenges ahead.
Rating: 3 (Neutral)

Title: "Is Apple Inc. (AAPL) Among the Best Warren Buffett Stock to Buy Right Now?"
Content: Discusses Apple's position as a strong investment option under Warren Buffett’s philosophy, implying positivity.
Rating: 4 (Moderate positive)

Title: "The Mag 7 Stocks Are No Longer Magnificent. Where the Charts Say They Go Next."
Content: Broad commentary on tech stocks, including Apple, with no clear sentiment focus on Apple itself.
Rating: 3 (Neutral)

Title: "Broadcom Shares Jump as AI Growth Fuels Upbeat Forecast"
Content: Primarily about Broadcom, with indirect relevance to Apple as a client company.
Rating: 3 (Neutral)

Title: "Major Breakthrough: New Certificates Propel Apple iPhone 16 into Indonesia"
Content: Highlights progress in regulatory certification for iPhone sales in Indonesia, indicating positive developments for Apple’s market expansion.
Rating: 4 (Moderate positive)

Title: "In Rotten Market, Top Funds Harvest Apple Stock — And 10 Others"
Content: Indicates institutional confidence in Apple stock despite broader market challenges, reflecting positivity.
Rating: 4 (Moderate positive)

Title: "Apple's iPhone 16 Still on Hold in Indonesia Despite Key Certification Win"
Content: Acknowledges regulatory progress but emphasizes ongoing hurdles for product launch in Indonesia.
Rating: 3 (Neutral)

Title: "Cloud AI Today - Cirrascale Launches Inference Cloud For Scalable AI Integration"
Content: Not related to Apple stock specifically but mentions general AI developments in the industry.
Rating: 3 (Neutral)

Title: "38% of Berkshire Hathaway's Portfolio Is Invested in These 3 Unstoppable Dividend Stocks"
Content: General commentary on Warren Buffett’s portfolio with no specific focus on Apple stock sentiment.
Rating: 3 (Neutral)

Overall Sentiment Rating for Apple Stock
The individual ratings suggest a mix of sentiments:

Positive/Moderate Positive Ratings (5 or 4): 8 items

Neutral Ratings (3): 8 items

Negative/Moderate Negative Ratings (1 or 2): 4 items

The overall sentiment leans toward moderately positive for Apple stock, reflecting optimism tempered by some concerns over delays and market challenges.

Final Overall Sentiment Rating:
4 (Moderate positive)
```


## [Cointegration](#table-of-contents)

ChatGPT is not able to provide the stock with highest cointegration value with Apple stock:

```
The query about which S&P 500 stock has the highest cointegration value with Apple stock cannot be answered directly using the provided search results. Cointegration analysis requires statistical computations based on historical price data of Apple stock and other S&P 500 stocks. This involves time-series analysis to determine the long-term equilibrium relationship between Apple and other stocks.

If you have access to historical price data for Apple and other S&P 500 stocks, you can calculate cointegration values using statistical software or programming languages such as Python or R. The process typically involves:

Collecting Data: Obtain historical price data for Apple (AAPL) and other S&P 500 stocks over a specific period.

Stationarity Testing: Use tests like the Augmented Dickey-Fuller (ADF) test to ensure the time series are stationary.

Cointegration Testing: Apply tests such as the Engle-Granger two-step method or Johansen test to identify stocks with high cointegration values with Apple.

If you provide relevant price data or specify tools available, I can guide you further on how to perform this analysis.
```


## [Correlation](#table-of-contents)

ChatGPT 4-o stated Microsoft has the highest degree of correlation:

```
The stock most correlated with Apple (AAPL) among S&P 500 companies is Microsoft (MSFT). Historically, Apple and Microsoft have shown a high degree of correlation due to their positions as leading technology companies and their exposure to similar market dynamics, such as consumer electronics, cloud computing, and software services.

While specific numerical correlation values for S&P 500 stocks are not provided in the search results, Microsoft is widely recognized as one of Apple’s closest peers in terms of performance trends and market behavior. If you need precise correlation coefficients, you would need to calculate them using historical price data for Apple and other S&P 500 stocks over a specific period. Tools like Python, R, or financial platforms can help generate a correlation matrix for this purpose.
```


# [Selenium](#table-of-contents)

- Selenium is not suitable as it occasionally run into ReadTimeoutError causing the code to break. Hence we are not able to perform scrolling to required level.
- Running in Jupyter Notebook causes timeout error due to async operation of Jupyter Notebook.


In [ ]:
url = "https://finance.yahoo.com/quote/AAPL/news/"
scroll_limit = 5

driver = webdriver.Chrome()
driver.get(url)

# Locate <a> tag and navigate to its parent <div>
parent_div = driver.find_elements(By.XPATH, "//div[@class='content yf-82qtw3']")

# parent_div.get_attribute("class")

# # Print the class attribute of the parent <div>
# title_element = parent_div.find_element(By.XPATH, ".//h3[contains(@class, 'yf-82qtw3)]")
# title_element.text

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=37797): Read timed out. (read timeout=120)

# [Playwright](#table-of-contents)


In [ ]:
<div class="" content yf-82qtw3><a class="subtle-link fin-size-small titles noUnderline yf-1xqzjha"
        data-ylk="elm:hdln;elmt:link;itc:0;ct:story;slk:Shareholders%20are%20showing%20signs%20of%20DEI%20fatigue%20as%20activists%20push%20for%20more%20votes;sec:qsp-news;subsec:all;cpos:1;g:a42c9cba-3739-4dae-8402-e5acaa0ad944"
        href="https://finance.yahoo.com/news/shareholders-are-showing-signs-of-dei-fatigue-as-activists-push-for-more-votes-140136546.html"
        aria-label="Shareholders are showing signs of DEI fatigue as activists push for more votes"
        title="Shareholders are showing signs of DEI fatigue as activists push for more votes" data-rapid_p="23"
        data-v9y="1">
        <h3 class="clamp  yf-82qtw3">Shareholders are showing signs of DEI fatigue as activists push for more votes</h3>
        <p class="clamp  yf-82qtw3">Champions and critics of DEI policies are again pushing American companies to either
            bolster or diminish their diversity policies. But so far this year, none of the proposals are garnering
            shareholder support.</p>
    </a>
    <div class="footer yf-82qtw3">
        <div class="publishing yf-1weyqlp">Yahoo Finance <i aria-hidden="true" data-svelte-h="svelte-sg6bkb">•</i> 13
            hours ago</div>
        <div class="taxonomy-links yf-13tszyc"><a data-testid="ticker-container"
                class="ticker x-small hover2 border streaming yf-1fqyif7" aria-label="DE"
                data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:DE;subsec:all;cpos:1" href="/quote/DE/" title="DE"
                data-rapid_p="24" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">DE </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="DE" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 c4af00a5">+3.48%</span></fin-streamer>
            </a> <a data-testid="ticker-container" class="ticker x-small hover2 border streaming yf-1fqyif7"
                aria-label="COST" data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:COST;subsec:all;cpos:1"
                href="/quote/COST/" title="COST" data-rapid_p="25" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">COST </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="COST" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 d5882186">-6.07%</span></fin-streamer>
            </a> <a data-testid="ticker-container" class="ticker x-small hover2 border streaming yf-1fqyif7"
                aria-label="AAPL" data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:AAPL;subsec:all;cpos:1"
                href="/quote/AAPL/" title="AAPL" data-rapid_p="26" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">AAPL </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="AAPL" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 c4af00a5">+1.59%</span></fin-streamer>
            </a> </div>
    </div>
</div>

In [2]:
df = pd.read_csv("./data/sentiment.csv")
df

,pub_date,ticker,publisher,period,title,content,prosusai,yiyanghkust,ziweichen,aventiq_ai
0,2025-03-11 22:10:00,AAPL,Yahoo Finance,1 hour ago,Trump's DOJ wants a Google breakup but is will...,President Trump and his predecessor Joe Biden ...,3,3,3,4
1,2025-03-11 05:10:00,AAPL,Yahoo Finance,18 hours ago,"Nvidia, Tesla slammed as 'Magnificent 7' names...","The ""Magnificent Seven"" stocks were at the cen...",1,1,1,1
2,2025-03-11 22:44:00,AAPL,Barrons.com,26 minutes ago,"These Stocks Are Moving the Most Today: Tesla,...",Tesla stock dips after tumbling 15% in Monday’...,1,1,1,1
3,2025-03-11 22:10:00,AAPL,TipRanks,1 hour ago,"DOJ ‘not calling off the dogs’ on Google, says...",After Google (GOOGL) and the Department of Jus...,2,3,3,1
4,2025-03-11 22:10:00,AAPL,Benzinga,1 hour ago,Is The Data Center REIT Honeymoon Period Over?,Data center REITs generated solid profits for ...,4,5,3,1
...,...,...,...,...,...,...,...,...,...,...
475,2024-12-11 23:11:00,PG,Reuters,3 months ago,"Exclusive-As tariffs loom, Gillette-razor make...",Procter & Gamble has overhauled its supply cha...,5,3,3,5
476,2024-12-11 23:11:00,PG,Zacks,3 months ago,Procter & Gamble (PG) Stock Slides as Market R...,"In the most recent trading session, Procter & ...",1,3,3,1
477,2024-12-11 23:11:00,PG,Motley Fool,3 months ago,3 Dividend King Stocks Near All-Time Highs Tha...,Paying up for quality companies can be a winni...,5,5,3,5
478,2024-12-11 23:11:00,PG,Business Wire,3 months ago,Braun IPLs Light Up Group Chats with a Major D...,"BOSTON, December 03, 2024--Group chats; love t...",3,3,3,3


In [ ]:
tickers = ["AAPL", "META", "GOOGL", "NVDA"]

df1 = yf.download(
    tickers, start="2025-01-01", end=None, group_by="ticker", rounding=True
)
df1

[*********************100%***********************]  4 of 4 completed


Ticker        META                                      AAPL                  \
Price         Open    High     Low   Close    Volume    Open    High     Low   
Date                                                                           
2025-01-02  589.72  604.91  587.82  599.24  12682300  248.66  248.83  241.55   
2025-01-03  604.76  609.50  596.41  604.63  11436800  243.09  243.91  241.62   
2025-01-06  611.83  630.99  605.62  630.20  14560800  244.04  247.06  242.93   
2025-01-07  631.70  632.10  608.23  617.89  12071500  242.71  245.28  241.08   
2025-01-08  613.40  616.44  602.79  610.72  10085800  241.65  243.44  239.79   
2025-01-10  610.03  629.91  597.34  615.86  19256600  239.75  239.90  232.74   
2025-01-13  607.10  608.97  598.45  608.33  10785000  233.27  234.41  229.47   
2025-01-14  605.07  605.49  588.55  594.25  13598000  234.49  235.86  232.21   
2025-01-15  610.21  622.46  606.50  617.12  15524300  234.38  238.70  234.17   
2025-01-16  611.50  616.25  609.71  611.30   8355300  237.09  237.75  227.78   
2025-01-17  624.05  624.97  603.67  612.77  17280600  231.87  232.03  228.23   
2025-01-21  617.83  621.50  609.01  616.46  11674000  223.75  224.17  219.14   
2025-01-22  623.30  633.70  619.43  623.50  12274000  219.55  223.87  219.55   
2025-01-23  627.36  636.60  621.15  636.45   9910200  224.49  226.78  222.06   
2025-01-24  636.40  652.00  634.20  647.49  19065300  224.53  225.38  221.17   
2025-01-27  626.61  663.87  625.10  659.88  28399200  223.77  231.90  223.73   
2025-01-28  666.00  682.58  659.47  674.33  20950100  230.60  239.93  230.56   
2025-01-29  675.00  681.05  665.67  676.49  21377800  233.86  239.60  233.75   
2025-01-30  696.82  710.79  676.10  687.00  29322900  238.41  240.53  236.95   
2025-01-31  691.00  705.30  685.58  689.18  19331700  246.92  246.92  233.18   
2025-02-03  675.91  704.00  675.25  697.46  21415700  229.74  231.58  225.45   
2025-02-04  702.50  707.71  691.61  704.19  13826300  227.00  232.87  226.40   
2025-02-05  703.55  718.14  699.01  704.87  17778200  228.28  232.41  228.02   
2025-02-06  705.88  718.90  703.50  711.99  13080700  231.04  233.54  230.18   
2025-02-07  716.80  725.01  711.75  714.52  16427100  232.34  233.74  227.01   
2025-02-10  718.56  721.20  711.33  717.40  12904300  229.57  230.59  227.20   
2025-02-11  713.32  723.66  710.04  719.80  12998000  228.20  235.23  228.13   
2025-02-12  715.30  727.10  712.60  725.38  12016500  231.20  236.96  230.68   
2025-02-13  721.52  729.00  718.04  728.56  12569100  236.91  242.34  235.57   
2025-02-14  726.14  740.91  725.62  736.67  16901200  241.25  245.55  240.99   
2025-02-18  736.00  737.00  706.44  716.37  21938700  244.15  245.18  241.84   
2025-02-19  704.10  707.00  695.39  703.77  17535500  244.66  246.01  243.16   
2025-02-20  697.28  705.63  693.64  694.84  12587400  244.94  246.78  244.29   
2025-02-21  696.58  703.87  682.36  683.55  15660400  245.95  248.69  245.22   
2025-02-24  686.28  687.27  662.45  668.13  15677000  244.93  248.86  244.42   
2025-02-25  665.97  668.00  641.86  657.50  20579700  248.00  250.00  244.91   
2025-02-26  659.65  683.01  658.00  673.70  14488700  244.33  244.98  239.13   
2025-02-27  682.45  688.65  657.57  658.24  12500000  239.41  242.46  237.06   
2025-02-28  658.04  669.63  642.60  668.20  17534200  236.95  242.09  230.20   
2025-03-03  673.68  681.25  650.84  655.05  10843800  241.79  244.03  236.11   
2025-03-04  645.51  649.75  623.65  640.00  21495200  237.71  240.07  234.68   
2025-03-05  641.00  659.50  637.66  656.47  13770000  235.42  236.55  229.23   
2025-03-06  648.00  650.56  624.11  627.93  13446500  234.44  237.86  233.16   
2025-03-07  625.37  634.79  600.61  625.66  21375700  235.11  241.37  234.76   
2025-03-10  608.07  610.11  586.87  597.99  21999900  235.54  236.16  224.22   
2025-03-11  595.05  614.93  593.33  605.71  17333300  223.81  225.84  217.45   

Ticker                          GOOGL                                    \
Pric

In [60]:
df1.stack(level=0, future_stack=True)

Price                Open    High     Low   Close     Volume
Date       Ticker                                           
2025-01-02 META    589.72  604.91  587.82  599.24   12682300
           AAPL    248.66  248.83  241.55  243.58   55740700
           GOOGL   190.43  191.78  187.28  189.21   20370800
           NVDA    136.00  138.88  134.63  138.31  198247200
2025-01-03 META    604.76  609.50  596.41  604.63   11436800
...                   ...     ...     ...     ...        ...
2025-03-10 NVDA    109.90  111.85  105.46  106.98  366487400
2025-03-11 META    595.05  614.93  593.33  605.71   17333300
           AAPL    223.81  225.84  217.45  220.84   76087700
           GOOGL   164.91  166.75  161.37  164.04   37720000
           NVDA    106.99  112.24  104.77  108.76  353708300

[184 rows x 5 columns]

In [6]:
df = yf.download(tickers, period="5d", group_by="Ticker")
df

[*********************100%***********************]  4 of 4 completed


Ticker           GOOGL                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2025-03-05  170.323845  173.580089  168.865517  172.820969  30954900   
2025-03-06  170.333820  174.608895  170.303856  172.151733  28302000   
2025-03-07  171.062988  174.768727  170.074137  173.660004  27385800   
2025-03-10  168.259995  168.460007  163.690002  165.869995  43604000   
2025-03-11  164.910004  166.750000  161.369995  164.039993  37720000   

Ticker            AAPL                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2025-03-05  235.419998  236.550003  229.229996  235.740005  47227600   
2025-03-06  234.440002  237.860001  233.160004  235.330002  45170400   
2025-03-07  235.110001  241.369995  234.759995  239.070007  46273600   
2025-03-10  235.539993  236.160004  224.220001  227.479996  72071200   
2025-03-11  223.809998  225.839996  217.449997  220.839996  76087700   

Ticker            NVDA                                                 \
Price             Open        High         Low       Close     Volume   
Date                                                                    
2025-03-05  117.580002  118.279999  114.510002  117.300003  284337900   
2025-03-06  113.529999  115.349998  110.220001  110.570000  321181900   
2025-03-07  111.250000  113.480003  107.559998  112.690002  341755500   
2025-03-10  109.900002  111.849998  105.459999  106.980003  366487400   
2025-03-11  106.989998  112.239998  104.769997  108.760002  353708300   

Ticker            META                                                
Price             Open        High         Low       Close    Volume  
Date                                                                  
2025-03-05  641.000000  659.500000  637.659973  656.469971  13770000  
2025-03-06  648.000000  650.559998  624.109985  627.929993  13446500  
2025-03-07  625.369995  634.789978  600.609985  625.659973  21375700  
2025-03-10  608.070007  610.109985  586.869995  597.989990  21999900  
2025-03-11  595.049988  614.929993  593.330017  605.710022  17333300

In [8]:
df.stack(level=0, future_stack=True)

Price                    Open        High         Low       Close     Volume
Date       Ticker                                                           
2025-03-05 GOOGL   170.323845  173.580089  168.865517  172.820969   30954900
           AAPL    235.419998  236.550003  229.229996  235.740005   47227600
           NVDA    117.580002  118.279999  114.510002  117.300003  284337900
           META    641.000000  659.500000  637.659973  656.469971   13770000
2025-03-06 GOOGL   170.333820  174.608895  170.303856  172.151733   28302000
           AAPL    234.440002  237.860001  233.160004  235.330002   45170400
           NVDA    113.529999  115.349998  110.220001  110.570000  321181900
           META    648.000000  650.559998  624.109985  627.929993   13446500
2025-03-07 GOOGL   171.062988  174.768727  170.074137  173.660004   27385800
           AAPL    235.110001  241.369995  234.759995  239.070007   46273600
           NVDA    111.250000  113.480003  107.559998  112.690002  341755500
           META    625.369995  634.789978  600.609985  625.659973   21375700
2025-03-10 GOOGL   168.259995  168.460007  163.690002  165.869995   43604000
           AAPL    235.539993  236.160004  224.220001  227.479996   72071200
           NVDA    109.900002  111.849998  105.459999  106.980003  366487400
           META    608.070007  610.109985  586.869995  597.989990   21999900
2025-03-11 GOOGL   164.910004  166.750000  161.369995  164.039993   37720000
           AAPL    223.809998  225.839996  217.449997  220.839996   76087700
           NVDA    106.989998  112.239998  104.769997  108.760002  353708300
           META    595.049988  614.929993  593.330017  605.710022   17333300

In [48]:
a = (
    df.stack(level=0, future_stack=True)
    .rename_axis(["Date", "Ticker"])
    .reset_index(level=1)
)
a

Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2025-03-05,GOOGL,170.323845,173.580089,168.865517,172.820969,30954900
2025-03-05,AAPL,235.419998,236.550003,229.229996,235.740005,47227600
2025-03-05,NVDA,117.580002,118.279999,114.510002,117.300003,284337900
2025-03-05,META,641.000000,659.500000,637.659973,656.469971,13770000
2025-03-06,GOOGL,170.333820,174.608895,170.303856,172.151733,28302000
2025-03-06,AAPL,234.440002,237.860001,233.160004,235.330002,45170400
2025-03-06,NVDA,113.529999,115.349998,110.220001,110.570000,321181900
2025-03-06,META,648.000000,650.559998,624.109985,627.929993,13446500
2025-03-07,GOOGL,171.062988,174.768727,170.074137,173.660004,27385800


In [ ]:
a = a.reset_index().sort_values(by=["Ticker", "Date"], ascending=True).set_index("Date")
# a.columns.name = None
a

Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2025-03-05,AAPL,235.419998,236.550003,229.229996,235.740005,47227600
2025-03-06,AAPL,234.440002,237.860001,233.160004,235.330002,45170400
2025-03-07,AAPL,235.110001,241.369995,234.759995,239.070007,46273600
2025-03-10,AAPL,235.539993,236.160004,224.220001,227.479996,72071200
2025-03-11,AAPL,223.809998,225.839996,217.449997,220.839996,76087700
2025-03-05,GOOGL,170.323845,173.580089,168.865517,172.820969,30954900
2025-03-06,GOOGL,170.333820,174.608895,170.303856,172.151733,28302000
2025-03-07,GOOGL,171.062988,174.768727,170.074137,173.660004,27385800
2025-03-10,GOOGL,168.259995,168.460007,163.690002,165.869995,43604000


In [50]:
a.columns.name = None
a

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2025-03-05,AAPL,235.419998,236.550003,229.229996,235.740005,47227600
2025-03-06,AAPL,234.440002,237.860001,233.160004,235.330002,45170400
2025-03-07,AAPL,235.110001,241.369995,234.759995,239.070007,46273600
2025-03-10,AAPL,235.539993,236.160004,224.220001,227.479996,72071200
2025-03-11,AAPL,223.809998,225.839996,217.449997,220.839996,76087700
2025-03-05,GOOGL,170.323845,173.580089,168.865517,172.820969,30954900
2025-03-06,GOOGL,170.333820,174.608895,170.303856,172.151733,28302000
2025-03-07,GOOGL,171.062988,174.768727,170.074137,173.660004,27385800
2025-03-10,GOOGL,168.259995,168.460007,163.690002,165.869995,43604000


In [14]:
ticker = yf.Ticker("AAPL")
ticker.news

[{'id': 'ee08c516-d812-48ab-9787-b73c34782bf7',
  'content': {'id': 'ee08c516-d812-48ab-9787-b73c34782bf7',
   'contentType': 'STORY',
   'title': "Wall Street's souring view on the US stock market isn't just about the economy — it's about AI too",
   'description': '',
   'summary': "As economic uncertainty has spawned a stock market sell-off, one of investors' favorite trades of the past two years is going through its own rerating of expectations.",
   'pubDate': '2025-03-11T17:44:49Z',
   'displayTime': '2025-03-11T17:44:49Z',
   'isHosted': True,
   'bypassModal': False,
   'previewUrl': None,
   'thumbnail': {'originalUrl': 'https://s.yimg.com/os/creatr-uploaded-images/2025-03/88bc1fd0-fb15-11ef-8dbd-ea4aea580f37',
    'originalWidth': 4500,
    'originalHeight': 3001,
    'caption': '',
    'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/CAOKVOjTmaybJSdSciq_vQ--~B/aD0zMDAxO3c9NDUwMDthcHBpZD15dGFjaHlvbg--/https://s.yimg.com/os/creatr-uploaded-images/2025-03/88bc1fd0-fb1

In [18]:
ticker.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-02-12 00:00:00-05:00,231.199997,236.960007,230.679993,236.869995,45243300,0.0,0.0
2025-02-13 00:00:00-05:00,236.910004,242.339996,235.570007,241.529999,53614100,0.0,0.0
2025-02-14 00:00:00-05:00,241.250000,245.550003,240.990005,244.600006,40896200,0.0,0.0
2025-02-18 00:00:00-05:00,244.149994,245.179993,241.839996,244.470001,48822500,0.0,0.0
2025-02-19 00:00:00-05:00,244.660004,246.009995,243.160004,244.869995,32204200,0.0,0.0
2025-02-20 00:00:00-05:00,244.940002,246.779999,244.289993,245.830002,32316900,0.0,0.0
2025-02-21 00:00:00-05:00,245.949997,248.690002,245.220001,245.550003,53197400,0.0,0.0
2025-02-24 00:00:00-05:00,244.929993,248.860001,244.419998,247.100006,51326400,0.0,0.0
2025-02-25 00:00:00-05:00,248.000000,250.000000,244.910004,247.039993,48013300,0.0,0.0


In [19]:
tickers = yf.Tickers("MSFT AAPL GOOG")
tickers.history()

[*********************100%***********************]  3 of 3 completed


Price            Close                         Dividends             \
Ticker            AAPL        GOOG        MSFT      AAPL GOOG  MSFT   
Date                                                                  
2025-02-12  236.869995  185.218979  408.221466       0.0  0.0  0.00   
2025-02-13  241.529999  187.666199  409.718475       0.0  0.0  0.00   
2025-02-14  244.600006  186.657349  407.612671       0.0  0.0  0.00   
2025-02-18  244.470001  185.588562  408.820282       0.0  0.0  0.00   
2025-02-19  244.869995  186.917053  413.940002       0.0  0.0  0.00   
2025-02-20  245.830002  186.427612  416.130005       0.0  0.0  0.83   
2025-02-21  245.550003  181.373367  408.209991       0.0  0.0  0.00   
2025-02-24  247.100006  180.983810  404.000000       0.0  0.0  0.00   
2025-02-25  247.039993  177.168152  397.899994       0.0  0.0  0.00   
2025-02-26  240.360001  174.501190  399.730011       0.0  0.0  0.00   
2025-02-27  237.300003  170.016312  392.529999       0.0  0.0  0.00   
2025-02-28  241.839996  172.024017  396.989990       0.0  0.0  0.00   
2025-03-03  238.029999  168.468079  388.489990       0.0  0.0  0.00   
2025-03-04  235.929993  172.413574  388.609985       0.0  0.0  0.00   
2025-03-05  235.740005  174.790878  401.019989       0.0  0.0  0.00   
2025-03-06  235.330002  174.011765  396.890015       0.0  0.0  0.00   
2025-03-07  239.070007  175.550003  393.309998       0.0  0.0  0.00   
2025-03-10  227.479996  167.809998  380.160004       0.0  0.2  0.00   
2025-03-11  220.839996  165.979996  380.450012       0.0  0.0  0.00   

Price             High                                 Low  ...              \
Ticker            AAPL        GOOG        MSFT        AAPL  ...        MSFT   
Date                                                        ...               
2025-02-12  236.960007  186.617395  409.928036  230.679993  ...  403.560798   
2025-02-13  242.339996  187.776074  410.177546  235.570007  ...  405.546817   
2025-02-14  245.550003  187.935891  408.091721  240.990005  ...  405.067786   
2025-02-18  245.179993  187.566304  409.778352  241.839996  ...  405.686551   
2025-02-19  246.009995  187.146787  414.658563  243.160004  ...  406.834255   
2025-02-20  246.779999  186.907062  419.309998  244.289993  ...  412.540009   
2025-02-21  248.690002  187.256664  418.049988  245.220001  ...  407.890015   
2025-02-24  248.860001  184.879366  409.369995  244.419998  ...  399.320007   
2025-02-25  250.000000  180.554293  401.920013  244.910004  ...  396.700012   
2025-02-26  244.979996  177.877349  403.600006  239.130005  ...  394.250000   
2025-02-27  242.460007  176.389041  405.739990  237.059998  ...  392.170013   
2025-02-28  242.089996  172.303697  397.630005  230.199997  ...  386.570007   
2025-03-03  244.029999  174.800860  398.820007  236.110001  ...  386.160004   
2025-03-04  240.070007  174.965659  392.579987  234.679993  ...  381.000000   
2025-03-05  236.550003  175.550008  401.670013  229.229996  ...  388.809998   
2025-03-06  237.860001  176.528886  402.149994  233.160004  ...  392.679993   
2025-03-07  241.369995  176.698688  394.799988  234.759995  ...  385.540009   
2025-03-10  236.160004  170.449997  386.399994  224.220001  ...  377.220001   
2025-03-11  225.839996  168.654999  386.000000  217.449997  ...  376.910004   

Price             Open                         Stock Splits            \
Ticker            AAPL        GOOG        MSFT         AAPL GOOG MSFT   
Date                                                                    
2025-02-12  231.199997  185.019210  406.395111          0.0  0.0  0.0   
2025-02-13  236.910004  185.718406  406.185551          0.0  0.0  0.0   
2025-02-14  241.250000  186.617401  406.973967          0.0  0.0  0.0   
2025-02-18  244.149994  187.226695  407.183549          0.0  0.0  0.0   
2025-02-19  244.660004  185.973121  407.063806          0.0  0.0  0.0   
2025-02-20  244.940002  186.287772  415.290009          0.0  0.0  0.0   
2025-02-21  245.949997  187.076861  417.3

In [33]:
a = np.arange(155)
batch_size = 20

for idx in range(0, len(a), batch_size):
    print(f"a[{idx} : {idx+batch_size}] : {a[idx : idx+batch_size]}")

a[0 : 20] : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
a[20 : 40] : [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
a[40 : 60] : [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59]
a[60 : 80] : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79]
a[80 : 100] : [80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]
a[100 : 120] : [100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119]
a[120 : 140] : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139]
a[140 : 160] : [140 141 142 143 144 145 146 147 148 149 150 151 152 153 154]


In [73]:
co_integrate = CoIntegrate()
co_integrate.run()

batch 1 : ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


[*********************100%***********************]  10 of 10 completed


batch 2 : ['APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL']


[*********************100%***********************]  10 of 10 completed


batch 3 : ['GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK']


...

batch 51 : ['ZTS']


[*********************100%***********************]  1 of 1 completed


len(df) :            Ticker    Open    High     Low   Close     Volume
Date                                                        
2020-01-02   ADBE  330.00  334.48  329.17  334.43  1990100.0
2020-01-03   ADBE  329.17  332.98  328.69  331.81  1577600.0
2020-01-06   ADBE  328.29  333.91  328.19  333.71  1874700.0
2020-01-07   ADBE  334.15  334.79  332.31  333.39  2500800.0
2020-01-08   ADBE  333.81  339.23  333.40  337.87  2248500.0
...           ...     ...     ...     ...     ...        ...
2025-03-05    WTW  331.94  336.28  331.94  334.50   629100.0
2025-03-06    WTW  333.03  333.16  328.89  330.43   636800.0
2025-03-07    WTW  330.46  332.05  327.14  330.89   463800.0
2025-03-10    WTW  330.50  335.87  327.52  332.00   508300.0
2025-03-11    WTW  333.52  334.19  325.22  326.52   578700.0

[653304 rows x 6 columns]


,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2020-01-02,ADBE,330.00,334.48,329.17,334.43,1990100.0
2020-01-03,ADBE,329.17,332.98,328.69,331.81,1577600.0
2020-01-06,ADBE,328.29,333.91,328.19,333.71,1874700.0
2020-01-07,ADBE,334.15,334.79,332.31,333.39,2500800.0
2020-01-08,ADBE,333.81,339.23,333.40,337.87,2248500.0
...,...,...,...,...,...,...
2025-03-05,WTW,331.94,336.28,331.94,334.50,629100.0
2025-03-06,WTW,333.03,333.16,328.89,330.43,636800.0
2025-03-07,WTW,330.46,332.05,327.14,330.89,463800.0


[*********************************************************************************************************************500%***********************************************************************************************************************]  5 of 1 completed

In [83]:
df = pd.read_parquet("./data/ohlcv_5y.parquet")
df

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2020-01-02,ADBE,330.00,334.48,329.17,334.43,1990100.0
2020-01-03,ADBE,329.17,332.98,328.69,331.81,1577600.0
2020-01-06,ADBE,328.29,333.91,328.19,333.71,1874700.0
2020-01-07,ADBE,334.15,334.79,332.31,333.39,2500800.0
2020-01-08,ADBE,333.81,339.23,333.40,337.87,2248500.0
...,...,...,...,...,...,...
2025-03-05,WTW,331.94,336.28,331.94,334.50,629100.0
2025-03-06,WTW,333.03,333.16,328.89,330.43,636800.0
2025-03-07,WTW,330.46,332.05,327.14,330.89,463800.0


In [84]:
df.isna().sum()

Ticker       0
Open      6846
High      6846
Low       6846
Close     6846
Volume    6846
dtype: int64

In [87]:
df_nan = df.loc[df["Open"].isna(), :]
df_nan["Ticker"].unique()

array(['ABNB', 'CARR', 'CEG', 'GEV', 'GEHC', 'KVUE', 'OTIS', 'PLTR', 'SW',
       'SOLV', 'VLTO'], dtype=object)

In [ ]:
a = df.loc[(df["Ticker"] == "CEG") & (~df["Open"].isna()), :]
a

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2022-01-19,CEG,36.88,41.73,36.88,40.76,129900.0
2022-01-20,CEG,39.74,46.94,39.74,43.67,153400.0
2022-01-21,CEG,41.73,48.48,41.68,43.67,329500.0
2022-01-24,CEG,43.59,43.67,40.64,41.54,110000.0
2022-01-25,CEG,41.25,43.59,40.11,40.66,23500.0
...,...,...,...,...,...,...
2025-03-05,CEG,230.22,233.10,223.00,228.89,3840200.0
2025-03-06,CEG,219.78,222.36,204.33,207.30,7710800.0
2025-03-07,CEG,208.10,216.14,203.10,212.54,4986200.0


In [ ]:
b = a.loc[(a["Open"] > 180) & (a["Open"] < 200), :]
b

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2024-03-07,CEG,180.51,180.81,175.01,178.36,1967800.0
2024-03-26,CEG,188.61,197.48,183.23,184.15,4228700.0
2024-03-27,CEG,186.80,188.83,181.10,183.63,2452100.0
2024-03-28,CEG,183.77,185.56,182.09,183.59,2050800.0
2024-04-01,CEG,184.76,187.53,182.15,184.71,1977500.0
...,...,...,...,...,...,...
2024-09-11,CEG,180.91,187.50,178.53,186.61,2846900.0
2024-09-12,CEG,186.36,191.02,183.56,189.28,2154900.0
2024-09-13,CEG,190.79,197.13,188.53,195.31,2613600.0


In [112]:
b.index

DatetimeIndex(['2024-03-07', '2024-03-26', '2024-03-27', '2024-03-28',
               '2024-04-01', '2024-04-02', '2024-04-03', '2024-04-04',
               '2024-04-05', '2024-04-08', '2024-04-09', '2024-04-10',
               '2024-04-11', '2024-04-12', '2024-04-15', '2024-04-16',
               '2024-04-17', '2024-04-18', '2024-04-19', '2024-04-22',
               '2024-04-23', '2024-04-24', '2024-04-25', '2024-04-26',
               '2024-04-29', '2024-04-30', '2024-05-01', '2024-05-02',
               '2024-05-03', '2024-05-06', '2024-05-07', '2024-05-08',
               '2024-06-10', '2024-07-18', '2024-07-19', '2024-07-22',
               '2024-07-23', '2024-07-24', '2024-07-31', '2024-08-01',
               '2024-08-07', '2024-08-08', '2024-08-09', '2024-08-12',
               '2024-08-13', '2024-08-14', '2024-08-15', '2024-08-16',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-26', '2024-08-27', '2024-08-28',
      

In [113]:
a.loc[b.index, ["Open"]]

,Open
Date,
2024-03-07,180.51
2024-03-26,188.61
2024-03-27,186.80
2024-03-28,183.77
2024-04-01,184.76
...,...
2024-09-11,180.91
2024-09-12,186.36
2024-09-13,190.79


In [114]:
len(a.index), len(b.index)

(788, 64)

In [126]:
c = set(a.index) & set(b.index)
a.loc[sorted(list(c)), :]

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2024-03-07,CEG,180.51,180.81,175.01,178.36,1967800.0
2024-03-26,CEG,188.61,197.48,183.23,184.15,4228700.0
2024-03-27,CEG,186.80,188.83,181.10,183.63,2452100.0
2024-03-28,CEG,183.77,185.56,182.09,183.59,2050800.0
2024-04-01,CEG,184.76,187.53,182.15,184.71,1977500.0
...,...,...,...,...,...,...
2024-09-11,CEG,180.91,187.50,178.53,186.61,2846900.0
2024-09-12,CEG,186.36,191.02,183.56,189.28,2154900.0
2024-09-13,CEG,190.79,197.13,188.53,195.31,2613600.0


In [57]:
df.loc[df["Ticker"] == "BRK.B", :]

,Ticker,Open,High,Low,Close,Volume,Adj Close
93888,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN
93889,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN
93890,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN
93891,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN
93892,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
95187,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN
95188,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN
95189,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN
95190,BRK.B,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df["Adj Close"].value_counts()

Series([], Name: count, dtype: int64)

In [67]:
yf.download("BF.B", start="2020-01-01")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-03-12)')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,BF.B,BF.B,BF.B,BF.B,BF.B,BF.B
Date,,,,,,


In [79]:
from itertools import combinations

a = ["a", "b", "c", "d", "e"]

for ticker1, ticker2 in combinations(a, 2):
    print(ticker1, ticker2)

a b
a c
a d
a e
b c
b d
b e
c d
c e
d e
